In [1]:
!pip install nltk


In [4]:
import random as r
from nltk import edit_distance #Расстояние Левинштейна, мера того, насколько строки отличаются
import json
import re
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer as cv
from sklearn.feature_extraction.text import TfidfVectorizer as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier



In [6]:
with open("big_bot_config.json", "r") as config_file:
    data = json.load(config_file)



In [7]:
INTENTS = data['intents']

In [8]:
def filter_text(text):
    text = text.lower()
    #text = text.lower().strip()
    #Убирать разные символы
    text = text.strip()
    #Найти все знаки преминания и заменить их на пустоту
    expression = r'[^\w\s]'
    text = re.sub(expression, "", text)
    #lambda t: re.sub(r'[^\w\s]', t.lower(), t.strip(),t) уточнить
    return text



def text_match(user_text, example):
    user_text = filter_text(user_text)
    example = filter_text(example)

    distance = edit_distance(user_text, example)
    avg_len = (len(user_text) + len(example)) / 2
    # Процент опечаток
    ratio = distance / avg_len
    if distance > 3:
        return False

    elif ratio > 0.4:
        return False

    else:
        return True
       #return ratio > 0.4

#print(text_match("Константинопольский!", "Констнтигопольокий"))

#Все намерения из карты
#print(INTENTS.keys())

#Определение намерений пользователя
def get_intent(user_text):
    for intent in INTENTS:
        examples = INTENTS[intent]["examples"]
        
        #func = [print(examples == filter_text(user_text)) for examples in INTENTS]
        for example in examples:
            if text_match(user_text, example):
                return intent
    
#print(get_intent("ПРВИЕТ ! ! !"))

#print(get_intent("привет"))
# СДелать случайный ответ из карты намерений
def get_random_response(intent):
    return r.choice(INTENTS[intent]['responses'])

# Классификация текстов




In [9]:
filter_text("ПРИВЕТ! Пр!   !")

#print(get_intent("расскажи анекдот!"))

'привет пр   '

In [10]:
# Подготовка данных
X = [] # Фраза (входные данные)
y = [] # Намерение (выходные данные)

for intent in INTENTS:
    examples = INTENTS[intent]['examples']
    for example in examples:
        text = filter_text(example)
        if len(text) < 3:
            continue
        else:
            X.append(text)
            y.append(intent)

# Векторизация текстов. Текст => вектор [0, 2, 5, 10]
# CountVectirizer, sklearn TfidfVectorizer


# Создаём векторайзер
vectorizer = cv()

# Обучить векторайзер
vectorizer.fit(X)

# Трансформация данных
vecX = vectorizer.transform(X) # сохраняем векторизованные тексты




In [11]:
model = RandomForestClassifier() # Настройки Добавить параметры
fit_model = model.fit(vecX, y) # Обучение

print(model.predict(vectorizer.transform(['Любимые занятия твои днём']))) # Сделать предсказание (по фразе определить намерение)

# Метрики
# Оценка качества:
y_pred = model.predict(vecX)

print('score_accuracy_score = ', accuracy_score(y, y_pred))
#print('score_r2_score = ', r2_score(y, y_pred, multioutput = '2500'))
print('score_recall_score = ', recall_score(y, y_pred, average = 'micro'))
#print('score_roc_curve = ', roc_curve(y, y_pred))
print('score_f1_score = ', f1_score(y, y_pred, average = 'macro'))


['wish']
score_accuracy_score =  0.829137294394092
score_recall_score =  0.829137294394092
score_f1_score =  0.7593533963506648


In [12]:
mlp = MLPClassifier()
mlp.fit(vecX,y)
mlp_pred = mlp.predict(vecX)


print('score_accuracy_score = ', accuracy_score(y, mlp_pred))
#print('score_r2_score = ', r2_score(y, y_pred, multioutput = '2500'))
print('score_recall_score = ', recall_score(y, mlp_pred, average = 'micro'))
#print('score_roc_curve = ', roc_curve(y, y_pred))
print('score_f1_score = ', f1_score(y, mlp_pred, average = 'macro'))



score_accuracy_score =  0.8288016112789527
score_recall_score =  0.8288016112789527
score_f1_score =  0.7502975498231754


/Users/macbook/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
# Определяем намерение с помощью модели: 
def get_intent_ml(text):
    vec_text = vectorizer.transform([text])
    pred_intent = model.predict(vec_text)[0]
    # model.predict_proba() < 30% ? => random.choice(faile_phrase) - определение вероятности других ответов
    # []
    return pred_intent

def bot(text):
    intent = get_intent(text)
    if intent:
        return get_random_response(intent)

    intent = get_intent_ml(text)
    return get_random_response(intent)

In [11]:
bot("расскажи анекдот")


'Сын спрашивает отца:\n — Пап, а правда, что все сказки начинаются: "В тридевятом царстве", или "Жили были старик со старухой"?\n — Нет, сынок! Некоторые сказки начинаются так: "Если вы меня изберете, то я вам..."'

In [12]:
# Поговорить с ботом

#user_text = ''
#while user_text != "exit":
#  print("[USER]: ", end = "")
#  user_text = input()
#  reply = bot(user_text)
#  print(f'[BOT]: {reply}')


In [13]:
#! pip install python-telegram-bot --pre
#! pip install nest-asyncio

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, filters

In [16]:
TOKEN = '5864216251:AAF-uP6r15oBpkRe3pzorX-jtqlGWeypHuo'

In [17]:
app = ApplicationBuilder().token(TOKEN).build()

In [ ]:
async def reply_by_bot(upd: Update, _ctx):
    text = upd.message.text
    print(f'[USER]: {text}')
    reply = bot(text)
    print(f'[BOT]: {reply}')
    upd.effective_user.full_name
    await upd.message.reply_text(reply)

handler = MessageHandler(filters.TEXT, reply_by_bot)
app.add_handler(handler)

app.run_polling(poll_interval = 1)
#app.run_webhook()


[USER]: Привет1
[BOT]: Ку
[USER]: Что делаешь?
[BOT]: Еще увидимся!
[USER]: что мне делать?
[BOT]: Иди поешь. Еда - это хорошо.
[USER]: а дальше?
[BOT]: Это действительно радостно
[USER]: Расскажи анекдот
[BOT]: Послематчевая пресс-конференция после игры Сербия-Россия 5: 0. 
Вопрос из зала:
 — Станислав Саламович, вам не кажется, что сборная России сегодня прилюдно занималась тем, чем Дзюба занимался приватно?
 — Нет, вы абсолютно не правы! Сегодня был полноценный акт!
[USER]: Как тебя зовут?
[BOT]: Алиса
[USER]: Пойдём гулять
[BOT]: Ваш успех радует
[USER]: Ладно пок
[BOT]: В следующий раз стучи
[USER]: хорошо
[BOT]: Всегда пожалуйста!
